In [5]:
import gspread
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
from tqdm import tqdm as tqdm
from pathlib import Path

In [6]:
gc = gspread.oauth(http_client=gspread.BackOffHTTPClient)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1075887934827-7udlaeu5hq61qa2gkl9pmvh1gj1ufm8j.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A42529%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=cOHM3wOFH4mrgu9tVBhZDS2tdkS7bB&access_type=offline
Ouverture dans une session de navigateur existante.


[1529520:1529520:0100/000000.561286:ERROR:zygote_linux.cc(664)] write: Relais brisé (pipe) (32)


In [7]:
FOLDER_ID = "16vFggD_RFkWa-sKZipSi1Ugh_br8zvNX"

In [177]:
files = gc.list_spreadsheet_files(folder_id=FOLDER_ID)
files = [f for f in files if f["name"].startswith("falc.mtl.")]
files = [f for f in files if not any(n in f["name"].lower() for n in ["sami"])]

In [178]:
def load_worksheet(sheet):
    sample_id = int(sheet.get("A1")[0][0])
    data = pd.DataFrame(sheet.get_all_records(head=2))
    data = data.iloc[:, 1:]
    data["A ne pas évaluer"] = data["A ne pas évaluer"].replace({'TRUE': True, 'FALSE': False}).astype(bool)
    data = data[~data["A ne pas évaluer"]]
    data.index = data.loc[:, "Code critère"]
    data = data.loc[:, "Respecté ?"]
    data["sample_id"] = sample_id
    
    return data

def load_eval_spreadsheet(file_id, annotator_id):
    file = gc.open_by_key(file_id)
    sheets = file.worksheets()
    sheets = [load_worksheet(s) for s in tqdm(sheets)]
    df = pd.DataFrame(sheets)
    df = df.set_index("sample_id")
    df.loc[:, "annotator_id"] = annotator_id
    df = df.set_index("annotator_id", append=True)
    return df

def load_all(files, local_dir, save=True, load=True):
    dfs = []
    for f in tqdm(files):
        annot = f["name"].split(".")[-1].lower()
        print(annot)
        try:    
            f_id = f["id"]
            local_dir.mkdir(exist_ok=True)
            path = local_dir / f"{annot}.csv"
            if load and path.exists():
                df = pd.read_csv(path)
            else:
                df = load_eval_spreadsheet(f_id, annot)
                if save:
                    df.to_csv(path)
            dfs.append(df)
        except ValueError as e :
            print(annot)
            print(e)
            
    return pd.concat(dfs)

In [179]:
interannot_files = [f for f in files if "interannot" in f["name"]]
eval_files = [f for f in files if "interannot" not in f["name"]]

In [135]:
# x = load_eval_spreadsheet(
#     "1j_FZzYPW8rgxZYwiaBwoEnyvGAxNUqIxPeIbbWuWTpA",
#     "azamat"
# )

In [199]:
eval_df = load_all(
    eval_files,
    Path("./data/annotated_df/eval/").resolve()
)
eval_df = eval_df.drop(eval_df.filter(regex="Unnamed").columns.tolist(), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 395.92it/s]

fabrice
azamat
dhia
tarik
alexis
jeremie
melanie
youssef
elisa


In [200]:
gen_df = pd.read_csv("./data/texts.csv", index_col=list(range(4)))
gen_df.index = gen_df.index.set_names(['sample_id'] + gen_df.index.names[1:])

In [201]:
gen_df.xs(656)

,,,inputs,value
model,test_set,input_id,,
"('llamaVIII', 'lora', 'etrfr')",test_etr_fr,21,Profitant d'un moment où les braconniers rient...,Lucas se faufile dans le camion des braconnier...


In [202]:
eval_df.annotator_id.unique()

array(['fabrice', 'azamat', 'dhia', 'tarik', 'alexis', 'jeremie',
       'melanie', 'youssef', 'elisa'], dtype=object)

In [203]:
(gen_df
    .reset_index(["model", "test_set", "input_id"])
)

,model,test_set,input_id,inputs,value
sample_id,,,,,
0,"('llamaVIIIinst', 'rag', 'etrfr+orangesum+wiki...",test_etr_fr_politic,0,"Travailleuses, travailleurs, En avril 2020, p...",Macron a dit en avril 2020 que le pays tenait ...
1,"('llamaVIIIinst', 'rag', 'etrfr+orangesum+wiki...",test_etr_fr_politic,1,PROFITS EN HAUSSE ET POUVOIR D’ACHAT EN BAISSE...,Les grandes entreprises sont de plus en plus r...
2,"('llamaVIIIinst', 'rag', 'etrfr+orangesum+wiki...",test_etr_fr_politic,2,AUGMENTATION DES SALAIRES ET DES PENSIONS! Les...,Les travailleurs et les retraités veulent des ...
3,"('llamaVIIIinst', 'rag', 'etrfr+orangesum+wiki...",test_etr_fr_politic,3,CONTRÔLE DES TRAVAILLEURS SUR LES COMPTES DES ...,Nous devons contrôler les comptes des grandes ...
4,"('llamaVIIIinst', 'rag', 'etrfr+orangesum+wiki...",test_etr_fr_politic,4,"HÔPITAUX, EHPAD: À BAS LA LOI DU PROFIT! 100 0...",La gestion comptable de la santé publique est ...
...,...,...,...,...,...
1801,"('mistVIIinst', 'rag', 'etrfr+orangesum+wikila...",test_etr_fr,48,– Tu me permets de rester dormir ici cette nui...,Bou demande à Jules s'il peut dormir chez lui ...
1802,"('mistVIIinst', 'rag', 'etrfr+orangesum+wikila...",test_etr_fr,49,Un intrus dans la maison – Maow… cela faisait ...,Un intrus a pénétré dans la maison. Bou se rév...
1803,"('mistVIIinst', 'rag', 'etrfr+orangesum+wikila...",test_etr_fr,50,"– J'ai fini la bouteille qui était au frais, i...",Le père de Jules a fini sa bouteille et demand...


In [204]:
df = (gen_df
    .reset_index(["model", "test_set", "input_id"])
    .merge(
        eval_df,
        left_on="sample_id", 
        right_on='sample_id'
    )
)
df

,sample_id,model,test_set,input_id,inputs,value,annotator_id,CI3,CI4,CI5,...,I1,I2,CA1,CA2,CA3,CA4,CA5,CA6,CA7,CA8
0,602,"('llamaVIII', 'lora', 'etrfr')",test_etr_fr_politic,0,"Travailleuses, travailleurs, En avril 2020, p...","Les travailleurs des hôpitaux et des Ehpad, le...",jeremie,Partiellement,Oui,Partiellement,...,Non,Non Applicable,Satisfaisant,Bon,Bon,Bon,Oui,Non,Non,Satisfaisant
1,605,"('llamaVIII', 'lora', 'etrfr')",test_etr_fr_politic,3,CONTRÔLE DES TRAVAILLEURS SUR LES COMPTES DES ...,Les travailleurs doivent contrôler les comptes...,elisa,Oui,Oui,Oui,...,Non,Non Applicable,Très bon,Très bon,Bon,Bon,Non,Non,Non,Bon
2,606,"('llamaVIII', 'lora', 'etrfr')",test_etr_fr_politic,4,"HÔPITAUX, EHPAD: À BAS LA LOI DU PROFIT! 100 0...",100 000 lits d’hôpitaux ont été supprimés en 2...,dhia,Oui,Oui,Oui,...,Non Applicable,Non Applicable,Bon,Bon,Bon,Bon,Non,Non,Non,Bon
3,607,"('llamaVIII', 'lora', 'etrfr')",test_etr_fr_politic,5,TRAVAILLEURS FRANÇAIS ET IMMIGRÉS: UN MÊME CAM...,Les travailleurs étrangers sont très utiles. S...,jeremie,Oui,Oui,Oui,...,Non Applicable,Non Applicable,Bon,Bon,Bon,Bon,Non,Non,Non,Très bon
4,608,"('llamaVIII', 'lora', 'etrfr')",test_etr_fr_politic,6,CONTRE LE SACCAGE DE LA PLANÈTE! Réchauffement...,CONTRE LE SACCAGE DE LA PLANÈTE! Les États ref...,youssef,Oui,Oui,Oui,...,Non,Oui,Très bon,Très bon,Très bon,Très bon,Non,Non,Non,Très bon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,1627,"('llamaVIII', 'mtllora', 'etrfr+orangesum+wiki...",test_etr_fr,46,Victoire! Jules n'arrive pas à s'endormir. Il ...,Jules n'arrive pas à s'endormir. Jules a peur ...,dhia,Oui,Oui,Oui,...,Non Applicable,Non Applicable,Bon,Bon,Bon,Bon,Non,Non,Non,Bon
212,1628,"('llamaVIII', 'mtllora', 'etrfr+orangesum+wiki...",test_etr_fr,47,"Non, on n'abandonne pas ses amis! Jules souffl...","« Non, on n'abandonne pas ses amis! » Jules so...",melanie,Oui,Oui,Oui,...,Non Applicable,Non Applicable,Bon,Bon,Bon,Bon,Partiellement,Non,Non,Satisfaisant
213,1630,"('llamaVIII', 'mtllora', 'etrfr+orangesum+wiki...",test_etr_fr,49,Un intrus dans la maison – Maow… cela faisait ...,Un intrus dans la maison – Bou et Jules se rév...,jeremie,Oui,Oui,Partiellement,...,Non Applicable,Non Applicable,Bon,Bon,Mauvais,Bon,Non,Non,Non,Satisfaisant
214,1631,"('llamaVIII', 'mtllora', 'etrfr+orangesum+wiki...",test_etr_fr,50,"– J'ai fini la bouteille qui était au frais, i...",Le père de Jules va chercher une autre bouteil...,tarik,Oui,Oui,Oui,...,Non Applicable,Non Applicable,Très bon,Très bon,Satisfaisant,Bon,Oui,Non Applicable,Non Applicable,Bon


In [205]:
df.model.unique()

array(["('llamaVIII', 'lora', 'etrfr')",
       "('llamaVIIIinst', 'rag', 'etrfr+wikilarge')",
       "('llamaVIIIinst', 'rag', 'etrfr')",
       "('llamaVIII', 'mtllora', 'etrfr+orangesum+wikilarge')"],
      dtype=object)

In [206]:
df.to_csv("./data/eval.csv")

In [207]:
df.columns[:7].to_list(),

(['sample_id',
  'model',
  'test_set',
  'input_id',
  'inputs',
  'value',
  'annotator_id'],)

In [120]:
criteria = ['CI3', 'CI4', 'CI5', 'CI6', 'CI8', 'CPM1',
       'CPM2', 'CPM3', 'CPM4', 'CPM5', 'CPM6', 'CPM7', 'CPM8', 'CPM9', 'CPM10',
       'CPM11', 'CPM12', 'CPM13', 'CPM14', 'CPM15', 'CPM16', 'CPM17', 'CPM18',
       'CPM19', 'CPM20', 'CPM21', 'I1', 'I2', 'CA1', 'CA2', 'CA3', 'CA4',
       'CA5', 'CA6', 'CA7', 'CA8']

# Melting the DataFrame
df_melted = df.melt(
    id_vars=df.columns[:7].to_list(),
    value_vars=criteria,
    # var_name='criteria',
    # value_name='value'
)

ValueError: value_name (value) cannot match an element in the DataFrame columns.

In [14]:
themes_crit = {
    "Choix de l'information": ['CI3', 'CI4', 'CI5', 'CI6', 'CI8'],
    "Choix des mots": ['CPM2', 'CPM3', 'CPM4',
       'CPM5', 'CPM6', 'CPM8', 'CPM9',  'CPM16', 'CPM17', 'CPM18', 'CPM19',
       'CPM20', 'CPM21'],
    "Construction des phrases": [
        'CPM1', "CPM7", "CPM10", "CPM11", "CPM12", 'CPM13', 'CPM14','CPM15',
    ],
    "Illustrations": ['I1', 'I2'],
    "Qualité globales": ['CA1', 'CA2', 'CA3', 'CA4',
       'CA5', 'CA6', 'CA7', 'CA8'],
}
crit_themes = {
    crit: k for k, v in themes_crit.items() for crit in v
}
type_crit = {
    "ETR": ['CI3', 'CI4', 'CI5', 'CI6', 'CI8', 'CPM1',
       'CPM2', 'CPM3', 'CPM4', 'CPM5', 'CPM6', 'CPM7', 'CPM8', 'CPM9', 'CPM10',
       'CPM11', 'CPM12', 'CPM13', 'CPM14', 'CPM15', 'CPM16', 'CPM17', 'CPM18',
       'CPM19', 'CPM20', 'CPM21', 'I1', 'I2'],
    "global quality": ['CA1', 'CA2', 'CA3', 'CA4',
       'CA5', 'CA6', 'CA7', 'CA8']
}
crit_type = {
    crit: k for k, v in type_crit.items() for crit in v
}
df_melted["criteria_theme"] = df_melted["criteria"].replace(crit_themes)
df_melted["criteria_type"] = df_melted["criteria"].replace(crit_type)
df_melted["annotator"] = df_melted["annotator"].replace({"elisa": "coder_1", "melanie": "coder_2", "stanislas":"coder_3"})

In [15]:
# df_melted = df_melted.pivot(index=['example_id', 'model', 'source', 'target', 'prediction', 
#                                    'dataset', 'sample_id', 'criteria', 'criteria_theme', 'criteria_type'],
#                            columns='annotator',
#                            values='value').reset_index()

In [16]:
crit_to_score = {
    'CI3': 2,
    'CI4': 6,
    'CI5': 2,
    'CI6': 2,
    'CI8': 2,
    'CPM1': 6,
    'CPM2': 6,
    'CPM3': 6,
    'CPM4': 2,
    'CPM5': 6,
    'CPM6': 4,
    'CPM7': 2,
    'CPM8': 2,
    'CPM9': 2,
    'CPM10': 4,
    'CPM11': 2,
    'CPM12': 4,
    'CPM13': 4,
    'CPM14': 4,
    'CPM15': 2,
    'CPM16': 2,
    'CPM17': 2,
    'CPM18': 2,
    'CPM19': 2,
    'CPM20': 2,
    'CPM21': 2,
    'I1': 4,
    'I2': 4
}
df_melted["criteria_score"] = df_melted["criteria"].replace(crit_to_score)

In [17]:
df_melted.to_csv(Path("./annot_tables/")/"results_melted.csv")

In [126]:
eval_df = load_all(
    interannot_files,
    Path("./data/annotated_df/interannot/").resolve()
)
eval_df = eval_df.drop(eval_df.filter(regex="Unnamed").columns.tolist(), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 277.98it/s]

dhia
azamat
tarik
alexis
jeremie
melanie
youssef
elisa


In [129]:
len(eval_df.groupby("sample_id"))

6

In [131]:
eval_df.to_csv("./data/interannot.csv")